In [2]:
import xgboost as xgb
from util import *
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2

/root/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


# Load Data

In [25]:
trainX, trainY, trainIdx, trainTimestamp, trainForward, trainEmbedding = \
    read_cache('CACHE/train_real_back3_forward1_slide15.h5')
    
cvX, cvY, cvIdx, cvTimestamp, cvForward, cvEmbedding = \
    read_cache('CACHE/cv_real_back3_forward1_slide15.h5')

In [26]:
trainX.shape, trainY.shape, trainIdx.shape, trainTimestamp.shape, trainForward.shape, trainEmbedding.shape

((10126, 3, 4), (10126, 1), (10126, 1), (10126,), (10126,), (10126, 8))

In [27]:
cvX.shape, cvY.shape, cvIdx.shape, cvTimestamp.shape, cvForward.shape, cvEmbedding.shape

((750, 3, 4), (750, 1), (750, 1), (750,), (750,), (750, 8))

In [31]:
# train = np.hstack((trainX.reshape((len(trainX), -1)), trainForward.reshape(-1, 1), trainEmbedding))
# cv = np.hstack((cvX.reshape((len(cvX), -1)), cvForward.reshape(-1, 1), cvEmbedding))
train = trainX.reshape((trainX.shape[0], -1))
cv = cvX.reshape((cvX.shape[0], -1))
dtrain = xgb.DMatrix(train, missing = -999.0, label= trainY)
dcv = xgb.DMatrix(cv, missing= -999.0, label=cvY)

watchlist = [ (dcv,'val'), (dtrain, 'train') ]


param = {'nthread': 8,
         'booster': 'gblinear',
         "objective": "reg:linear",
         "eval_metric": "rmse", "eta":  0.05,
         "max_depth": 6,
         "min_child_weight": 1,
         "gamma": 0.1,
         "lambda": 0.5,
         "alpha": 0.5,
         "scale_pos_weight": 1,
         "subsample": 1,
         'early_stopping_rounds':100}

num_round = 200
bst = xgb.train(param, dtrain, num_round, evals=watchlist)

bst.save_model("MODEL/xgb")

# test = np.load("data_fs/test.npy")



[0]	val-rmse:0.208776	train-rmse:0.241985
[1]	val-rmse:0.183591	train-rmse:0.19553
[2]	val-rmse:0.179554	train-rmse:0.1743
[3]	val-rmse:0.179679	train-rmse:0.163902
[4]	val-rmse:0.179301	train-rmse:0.157711
[5]	val-rmse:0.177678	train-rmse:0.153152
[6]	val-rmse:0.175298	train-rmse:0.149333
[7]	val-rmse:0.172348	train-rmse:0.145864
[8]	val-rmse:0.169142	train-rmse:0.142625
[9]	val-rmse:0.165848	train-rmse:0.13956
[10]	val-rmse:0.162543	train-rmse:0.136636
[11]	val-rmse:0.159364	train-rmse:0.13386
[12]	val-rmse:0.156273	train-rmse:0.131211
[13]	val-rmse:0.153319	train-rmse:0.128685
[14]	val-rmse:0.1505	train-rmse:0.126278
[15]	val-rmse:0.147845	train-rmse:0.12399
[16]	val-rmse:0.145314	train-rmse:0.121808
[17]	val-rmse:0.1429	train-rmse:0.119725
[18]	val-rmse:0.140638	train-rmse:0.117749
[19]	val-rmse:0.138496	train-rmse:0.115868
[20]	val-rmse:0.136444	train-rmse:0.114068
[21]	val-rmse:0.134512	train-rmse:0.112356
[22]	val-rmse:0.132699	train-rmse:0.110731
[23]	val-rmse:0.13098	train-rms

In [32]:
predict= bst.predict(dcv).reshape(-1, 1)
index = cvY[:, 0]>0
evaluate(predict[index], cvY[index], cvIdx[index], cvTimestamp[index], cvForward[index])

cv mape: 0.668333411217


0.66833341